In [1]:
!pip install gurobipy
import numpy as np
from gurobipy import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.5 MB 6.3 MB/s 


Khởi tạo mô hình và trọng số

In [2]:
m = Model()
d = np.array([
    [9, 6, 7, 2, 8],
    [6, 7, 4, 5, 3],
    [5, 2, 7, 2, 8],
    [9, 10, 7, 8, 2],
    [6, 11, 2, 9, 3],
    [9, 8, 7, 6, 2],
    [4, 5, 4, 3, 5],
    [13, 10, 11, 6, 6],
    [2, 3, 6, 7, 7],
])
d_base = len(d)
d_outpost = len(d[1,:])

Restricted license - for non-production use only - expires 2023-10-25


Khai báo biến

In [3]:
F = [i + 1 for i in range(d_base)]
N = [j + 1 for j in range(d_outpost)]
A = [(i, j) for i in F for j in N]

x = m.addVars(F, vtype = GRB.BINARY, name = 'x')
y = m.addVars(A, vtype = GRB.CONTINUOUS, lb = 0, name = 'y')

## Hàm mục tiêu

In [4]:
m.setObjective(quicksum(d[i - 1, j - 1]*y[i, j] for i, j in A), GRB.MAXIMIZE)

Khai báo các điều kiện ràng buộc

In [5]:
m.addConstrs(quicksum(y[i, j] for j in N) == 1 for i in F)
m.addConstr(quicksum(x[j] for j in N) == 2)
m.addConstrs(quicksum(y[i, k] for k in N if d[i - 1, k - 1] > d[i - 1, j - 1]) <= x[j] for (i, j) in A)
m.update()

In [6]:
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 55 rows, 54 columns and 180 nonzeros
Model fingerprint: 0xd5232552
Variable types: 45 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 27 rows and 29 columns
Presolve time: 0.00s
Presolved: 28 rows, 25 columns, 91 nonzeros
Variable types: 20 continuous, 5 integer (5 binary)
Found heuristic solution: objective 39.0000000

Root relaxation: objective 4.560000e+01, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   45.60000    0    5   39.00000   45.60000  16.9%     -    0s
H    0     0                      40.0000000   

In [7]:
m.printAttr('X')


    Variable            X 
-------------------------
        x[4]            1 
        x[5]            1 
      y[1,2]            1 
      y[2,3]            1 
      y[3,2]            1 
      y[4,3]            1 
      y[5,3]            1 
      y[6,3]            1 
      y[7,3]            1 
      y[8,2]            1 
      y[9,1]            1 
